In [1]:
import numpy as np  
import pandas as pd  
import os
import sys
sys.path.append("/kaggle/input/nfl-data/nfl_inference_scripts/nfl_inference_scripts")

**XGB preprocessing step**

In [2]:
!python /kaggle/input/nfl-data/nfl_inference_scripts/nfl_inference_scripts/xgb_preprocess.py

47330it [00:05, 8514.51it/s]
(7414, 17)
7414it [00:00, 11600.71it/s]
(7414, 17)
7414it [00:00, 7698.26it/s]
4312it [00:01, 2831.55it/s]


In [3]:
!python /kaggle/input/nfl-data/nfl_inference_scripts/nfl_inference_scripts/xgb_pair_v1.py

(7163, 12)
7163it [00:05, 1414.88it/s]


**Install libraries for mmaction2 model**

In [4]:
!pip install /kaggle/input/nfl-libs/nfl_libs/addict-2.4.0-py3-none-any.whl --no-deps
!pip install /kaggle/input/nfl-libs/nfl_libs/yapf-0.33.0-py2.py3-none-any.whl --no-deps
!pip install /kaggle/input/nfl-libs/nfl_libs/pytest_runner-6.0.0-py3-none-any.whl --no-deps
# !pip install /kaggle/input/nfl-libs/nfl_libs/mmengine-0.7.3-py3-none-any.whl --no-deps
!pip install /kaggle/input/nfl-libs/nfl_libs/mmcv_full-1.7.1-py2.py3-none-any.whl --no-deps

Processing /kaggle/input/nfl-libs/nfl_libs/addict-2.4.0-py3-none-any.whl
Processing /kaggle/input/nfl-libs/nfl_libs/yapf-0.33.0-py2.py3-none-any.whl
  Attempting uninstall: yapf
    Found existing installation: yapf 0.32.0
    Uninstalling yapf-0.32.0:
      Successfully uninstalled yapf-0.32.0
Processing /kaggle/input/nfl-libs/nfl_libs/pytest_runner-6.0.0-py3-none-any.whl
Processing /kaggle/input/nfl-libs/nfl_libs/mmcv_full-1.7.1-py2.py3-none-any.whl


In [5]:
import torch
from model_csn1 import NModel
import cv2
import albumentations as A
from tqdm.notebook import tqdm
import time
import gc

/opt/conda/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


**Crop endzone, sideline for player-player function**

In [6]:
def get_sample(vid, fr_id, idx1, idx2, window_size = 20, out_size = 128):
    ws = []
    hs = []
    for fr in range(fr_id + window_size[0], fr_id + window_size[-1] + 1):
        if fr in det_dict[vid]:
            if idx1 in det_dict[vid][fr] and idx2 in det_dict[vid][fr]: 
                x, y, w, h = det_dict[vid][fr][idx1]['box']
                ws.append(w)
                hs.append(h)
                x, y, w, h = det_dict[vid][fr][idx2]['box']
                ws.append(w)
                hs.append(h)

    if len(ws)>0:
        crop_size = int(4*max(np.mean(ws), np.mean(hs)))# 5*...
    else:
        crop_size = out_size

    bboxes = []
    for fr in range(fr_id + window_size[0], fr_id + window_size[-1] + 1):
        if fr in det_dict[vid]:
            if idx1 in det_dict[vid][fr] and idx2 in det_dict[vid][fr]: 
                x, y, w, h = det_dict[vid][fr][idx1]['box']
                x1 = x + w/2
                y1 = y + h/2

                x, y, w, h = det_dict[vid][fr][idx2]['box']
                x2 = x + w/2
                y2 = y + h/2

                xc = 0.5*x1 + 0.5*x2
                yc = 0.5*y1 + 0.5*y2

                bboxes.append([xc-crop_size, yc-crop_size, xc+crop_size, yc+crop_size])
            else:
                bboxes.append([np.nan, np.nan, np.nan, np.nan])
        else:
            bboxes.append([np.nan, np.nan, np.nan, np.nan])

    bboxes = pd.DataFrame(bboxes).interpolate(limit_direction='both').values
    images = []
    masks1 = []
    masks2 = []
    empty_count = 0
    
    if bboxes.sum() > 0:    
        for i, ii in enumerate(window_size):
            fr = ii + fr_id
            path = f'{vid}_{fr}'
            
            if path in image_dict:
                image = image_dict[path]
            else:
                image = np.ones((720, 1280,3), dtype = np.uint8)
                empty_count +=1

            mask1 = np.ones((720, 1280), dtype = np.uint8)
            mask2 = np.ones((720, 1280), dtype = np.uint8)

            x1, y1, x2, y2 = list(map(int, bboxes[ii-window_size[0]]))

            y1 = y1 + int(0.2*crop_size)
            x2 = x1 + crop_size*2
            y2 = y1 + crop_size*2

            if fr in det_dict[vid]:
                if idx1 in det_dict[vid][fr]: 
                    x, y, w, h = det_dict[vid][fr][idx1]['box']
                    # mask1[y:y+h, x:x+w] = 255
                    cv2.circle(mask1, (x+w//2, y+h//2), int(0.3*h+0.3*w), 255, thickness=-1)# int(0.25*h+0.25*w)

                if idx2 in det_dict[vid][fr]:
                    x, y, w, h = det_dict[vid][fr][idx2]['box']
                    # mask2[y:y+h, x:x+w] = 255
                    cv2.circle(mask2, (x+w//2, y+h//2), int(0.25*h+0.25*w), 255, thickness=-1)

            crop = image[y1:y2, x1:x2]
            crop_mask1 = mask1[y1:y2, x1:x2]
            crop_mask2 = mask2[y1:y2, x1:x2]

            cr_y, cr_x = crop.shape[:2]
            if cr_x == crop_size*2 and cr_y == crop_size*2:
                crop = cv2.resize(crop, (out_size*2,out_size*2))
                crop_mask1 = cv2.resize(crop_mask1, (out_size*2,out_size*2))
                crop_mask2 = cv2.resize(crop_mask2, (out_size*2,out_size*2))
                images.append(crop)
                masks1.append(crop_mask1)
                masks2.append(crop_mask2)
            else:
                tmp_crop =  np.ones((crop_size*2, crop_size*2,3), dtype = np.uint8)
                tmp_mask1 =  np.ones((crop_size*2, crop_size*2), dtype = np.uint8)
                tmp_mask2 =  np.ones((crop_size*2, crop_size*2), dtype = np.uint8)
                if x1 < 0:
                    if y2>=720:
                        tmp_crop[crop_size*2-cr_y:,:cr_x] = crop
                        tmp_mask1[crop_size*2-cr_y:,:cr_x] = crop_mask1
                        tmp_mask2[crop_size*2-cr_y:,:cr_x] = crop_mask2
                    else:
                        tmp_crop[:cr_y,:cr_x] = crop
                        tmp_mask1[:cr_y,:cr_x] = crop_mask1
                        tmp_mask2[:cr_y,:cr_x] = crop_mask2

                elif x2> 1280:
                    if y2>=720:
                        tmp_crop[crop_size*2-cr_y:,crop_size*2-cr_x:] = crop
                        tmp_mask1[crop_size*2-cr_y:,crop_size*2-cr_x:] = crop_mask1
                        tmp_mask2[crop_size*2-cr_y:,crop_size*2-cr_x:] = crop_mask2
                    else:
                        tmp_crop[:cr_y,crop_size*2-cr_x:] = crop
                        tmp_mask1[:cr_y,crop_size*2-cr_x:] = crop_mask1
                        tmp_mask2[:cr_y,crop_size*2-cr_x:] = crop_mask2

                tmp_crop = cv2.resize(tmp_crop, (out_size*2,out_size*2))
                tmp_mask1 = cv2.resize(tmp_mask1, (out_size*2,out_size*2))
                tmp_mask2 = cv2.resize(tmp_mask2, (out_size*2,out_size*2))
                images.append(tmp_crop)
                masks1.append(tmp_mask1)
                masks2.append(tmp_mask2)
    else:
        for i, ii in enumerate(window_size):            
            empty_count +=1
            crop =  np.ones((out_size*2, out_size*2,3), dtype = np.uint8)
            crop_mask1 =  np.ones((out_size*2, out_size*2), dtype = np.uint8)
            crop_mask2 =  np.ones((out_size*2, out_size*2), dtype = np.uint8)
            images.append(crop)
            masks1.append(crop_mask1)
            masks2.append(crop_mask2)
            
    return images, masks1, masks2

**Crop endzone, sideline for player-ground function**


In [7]:
def get_sample_G(vid, fr_id, idx1, idx2, window_size = 20, out_size = 128, scale=4):
    ws = []
    hs = []
    for fr in range(fr_id + window_size[0], fr_id + window_size[-1] + 1):
        if fr in det_dict[vid]:
            if idx1 in det_dict[vid][fr]: 
                x, y, w, h = det_dict[vid][fr][idx1]['box']
                ws.append(w)
                hs.append(h)
    
    if len(ws)>0:
        crop_size = int(scale*max(np.mean(ws), np.mean(hs)))
    else:
        crop_size = out_size

    bboxes = []
    for fr in range(fr_id + window_size[0], fr_id + window_size[-1] + 1):
        if fr in det_dict[vid]:
            if idx1 in det_dict[vid][fr]: 
                x, y, w, h = det_dict[vid][fr][idx1]['box']
                xc = x + w/2
                yc = y + h/2

                bboxes.append([xc-crop_size, yc-crop_size, xc+crop_size, yc+crop_size])
            else:
                bboxes.append([np.nan, np.nan, np.nan, np.nan])
        else:
            bboxes.append([np.nan, np.nan, np.nan, np.nan])

    bboxes = pd.DataFrame(bboxes).interpolate(limit_direction='both').values
    images = []
    masks1 = []
    empty_count = 0
    if bboxes.sum() > 0:
        for i, ii in enumerate(window_size):        
            fr = ii + fr_id
            path = f'{vid}_{fr}'

            if path in image_dict:
                if scale != 4:
                    image = image_dict1[path]
                    p_path = f'{vid}_{fr-2}'
                    if p_path in image_dict1:
                        p_img = image_dict1[p_path]
                    else:
                        p_img = image

                    n_path = f'{vid}_{fr+2}'
                    if n_path in image_dict1:
                        n_img = image_dict1[n_path]
                    else:
                        n_img = image

                    image = np.stack([p_img, image, n_img], -1) #.transpose(1,2,0)
                else:
                    image = image_dict[path]
            else:
                image = np.ones((720, 1280,3), dtype = np.uint8)
                empty_count +=1

            mask1 = np.ones((720, 1280), dtype = np.uint8)

            x1, y1, x2, y2 = list(map(int, bboxes[ii-window_size[0]]))

            y1 = y1 + int(0.2*crop_size)
            x2 = x1 + crop_size*2
            y2 = y1 + crop_size*2

            if fr in det_dict[vid]:
                if idx1 in det_dict[vid][fr]: 
                    x, y, w, h = det_dict[vid][fr][idx1]['box']
                    # cv2.rectangle(image, (x, y), (x+w, y+h), (0,0,255), 2)
                    # mask1[y:y+h, x:x+w] = 255
                    cv2.circle(mask1, (x+w//2, y+h//2), int(0.3*h+0.3*w), 255, thickness=-1)


            crop = image[y1:y2, x1:x2]
            crop_mask1 = mask1[y1:y2, x1:x2]

            cr_y, cr_x = crop.shape[:2]
            if cr_x == crop_size*2 and cr_y == crop_size*2:
                crop = cv2.resize(crop, (out_size*2,out_size*2))
                crop_mask1 = cv2.resize(crop_mask1, (out_size*2,out_size*2))
                images.append(crop)
                masks1.append(crop_mask1)
            else:
                tmp_crop =  np.ones((crop_size*2, crop_size*2,3), dtype = np.uint8)
                tmp_mask1 =  np.ones((crop_size*2, crop_size*2), dtype = np.uint8)
                if x1 < 0:
                    if y2>=720:
                        tmp_crop[crop_size*2-cr_y:,:cr_x] = crop
                        tmp_mask1[crop_size*2-cr_y:,:cr_x] = crop_mask1
                    else:
                        tmp_crop[:cr_y,:cr_x] = crop
                        tmp_mask1[:cr_y,:cr_x] = crop_mask1

                elif x2> 1280:
                    if y2>=720:
                        tmp_crop[crop_size*2-cr_y:,crop_size*2-cr_x:] = crop
                        tmp_mask1[crop_size*2-cr_y:,crop_size*2-cr_x:] = crop_mask1
                    else:
                        tmp_crop[:cr_y,crop_size*2-cr_x:] = crop
                        tmp_mask1[:cr_y,crop_size*2-cr_x:] = crop_mask1

                tmp_crop = cv2.resize(tmp_crop, (out_size*2,out_size*2))
                tmp_mask1 = cv2.resize(tmp_mask1, (out_size*2,out_size*2))
                images.append(tmp_crop)
                masks1.append(tmp_mask1)
    else:
        for i, ii in enumerate(window_size):        
            empty_count +=1
            # crop =  np.ones((crop_size*2, crop_size*2,3))
            crop =  np.ones((out_size*2, out_size*2, 3), dtype = np.uint8)
            crop_mask1 =  np.ones((out_size*2, out_size*2), dtype = np.uint8)
            images.append(crop)
            masks1.append(crop_mask1)
    return images, masks1, empty_count

**Test dataset**

In [8]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, df, image_dict, tfms=None, cfg=None):
        self.df = df.reset_index(drop=True)
        self.image_dict = image_dict
        self.transform = A.ReplayCompose([ A.Resize(cfg.img_size, cfg.img_size, interpolation=cv2.INTER_AREA, p=1.),])
        self.cfg = cfg
        
        self.trk_step = 0

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        row = self.df.loc[index]
        idx1 = int(row['nfl_player_id_1'])
        if self.cfg.is_G:
            idx2 = row['nfl_player_id_2']
        else:
            idx2 = int(row['nfl_player_id_2'])
        fr_id = int(row['frame'])
        step = int(row['step'])
        
        if self.cfg.is_G:
            e_images, e_masks1, _ = get_sample_G(e_vid, fr_id, idx1, idx2, window_size = cfg.window_size, scale=cfg.scale)
            s_images, s_masks1, _ = get_sample_G(s_vid, fr_id, idx1, idx2, window_size = cfg.window_size, out_size = 128, scale=cfg.scale)
        else:
            e_images, e_masks1, e_masks2 = get_sample(e_vid, fr_id, idx1, idx2, window_size = cfg.window_size)
            s_images, s_masks1, s_masks2 = get_sample(s_vid, fr_id, idx1, idx2, window_size = cfg.window_size, out_size = 128)
        
        images_e = []
        images_s = []
        for ii in range(len(e_images)):
            e_img = e_images[ii]
            e_img[e_masks1[ii]>100] = 255
            if not  self.cfg.is_G:
                e_img[e_masks2[ii]>100] = 0

            s_img = s_images[ii]
            s_img[s_masks1[ii]>100] = 255
            if not self.cfg.is_G:
                s_img[s_masks2[ii]>100] = 0

            images_e.append(e_img)
            images_s.append(s_img)

        e_images = np.array(images_e)
        s_images = np.array(images_s)
                
        num_empty = 0
        for img in e_images:
            h, w, c = img.shape 
            if np.sum(img<2)/(h*w*c) > 0.9:
                num_empty += 1
        if len(e_images) - num_empty < 2:
            # print('e empty')
            e_images = s_images.copy()

        num_empty = 0
        for img in s_images:
            h, w, c = img.shape 
            if np.sum(img<2)/(h*w*c) > 0.9:
                num_empty += 1
        if len(s_images) - num_empty < 2:
            # print('s empty')
            s_images = e_images.copy()            
        
        if self.trk_step == 0:
            self.trk_step = len(e_images)//2

        if not self.cfg.is_G:
            trk_images = self.render_trk(vid, step, idx1, idx2)
            
        images = []
        for i in range(len(s_images)):
            if not self.cfg.is_G:
                trk_img = trk_images[i]
                img = np.hstack([e_images[i], trk_img, s_images[i]])
            else:
                img = np.hstack([e_images[i], s_images[i]])
            images.append(img)
        img = np.array(images)
        img = img.transpose(3,0,1,2) #C T H W

        img = img/255.

        return torch.tensor(img, dtype=torch.float)
    
    def render_trk(self, vid, step, idx1, idx2):
        # simulate tracking data as image 128x256x3
        shift_x = 0
        shift_y = 0

        d_x = 5
        scale = 60/d_x

        idx = f'{vid}_{step}'
        images = []
        x1 = trk_dict[idx][idx1]['x']
        y1 = trk_dict[idx][idx1]['y']

        x2 = trk_dict[idx][idx2]['x']
        y2 = trk_dict[idx][idx2]['y']

        xc = 0.5*x1 + 0.5*x2
        yc = 0.5*y1 + 0.5*y2

        for st in range(step-self.trk_step, step + self.trk_step + 1):
            #print('=======',step, self.trk_step)
            this_idx = f'{vid}_{st}'
            img = np.ones((3, self.cfg.img_size, 128), dtype=np.uint8)
            if this_idx in trk_dict:
                for p_id, meta in trk_dict[this_idx].items():
                    x = meta['x']
                    y = meta['y']
                    t = meta['t']

                    x = x - xc + d_x
                    y = y - yc + (2*d_x)

                    x = round(x*scale) + shift_x
                    y = round(y*scale) + shift_y

                    if x>0 and y>0 and x<128 and y<self.cfg.img_size:
                        radius = 3
                        val = 125
                        if p_id in [idx1, idx2]:
                            radius = 5
                            val = 255

                        cv2.circle(img[0], (x, y), radius, val, thickness=-1)
                        if t == 'home':
                            cv2.circle(img[1], (x, y), radius, val, thickness=-1)
                        else:
                            cv2.circle(img[2], (x, y), radius, val, thickness=-1)


            img = img.transpose(1,2,0)
            images.append(img)

        return images

**Function to run inference for player-player contact**

In [9]:
def predict_pair(window_size, df_p, models):
    if df_p.shape[0] == 0:
        return df_p, 0
    cfg.is_G = 0
    cfg.window_size = window_size
    
    test_dataset = TestDataset(df_p, image_dict, cfg=cfg)
    test_loader = torch.utils.data.DataLoader(test_dataset,
            batch_size=8,#8
            num_workers=4,#4
            shuffle=False)
    
    y_preds = []
    with torch.no_grad():
        bar = tqdm(test_loader)
        for batch_idx, images in enumerate(bar):
            images = images.float().to(device)
            for ii, model in enumerate(models):
                with torch.cuda.amp.autocast():
                    if ii%2==0:
                        pred = model(images)
                    else:
                        pred = model(images.flip(-1))
                    logit = pred['out1'].sigmoid().detach().cpu().numpy()
                if ii == 0:
                    out = logit
                else:
                    out += logit
            out = out/len(models)
            y_preds.append(out)
    y_preds = np.concatenate(y_preds)
    print(y_preds.shape, df_p.shape)
    df_p['pred'] = y_preds
    return df_p, 1

**Function to run inference for player-ground contact**

In [10]:
def predict_g(window_size, df_g, models, scale):
    if df_g.shape[0] == 0:
        return df_g, 0
    cfg.is_G = 1
    cfg.window_size = window_size
    cfg.scale = scale 
    
    test_dataset = TestDataset(df_g, image_dict, cfg=cfg)
    test_loader = torch.utils.data.DataLoader(test_dataset,
            batch_size=8,#8
            num_workers=4,#4
            shuffle=False)
    y_preds = []
    with torch.no_grad():
        bar = tqdm(test_loader)
        for batch_idx, images in enumerate(bar):
            images = images.float().to(device)
            for ii, model in enumerate(models):
                with torch.cuda.amp.autocast():
                    if ii%2==0:
                        pred = model(images)
                    else:
                        pred = model(images.flip(-1))
                    logit = pred['out1'].sigmoid().detach().cpu().numpy()
                if ii == 0:
                    out = logit
                else:
                    out += logit
            out = out/len(models)
            y_preds.append(out)
    y_preds = np.concatenate(y_preds)
    print(y_preds.shape, df_g.shape)
    df_g['pred'] = y_preds
    return df_g, 1

**Load trained models**

In [11]:
class config:
    model_name = 'r50ir' 
    pool_type = 'avg'
    is_G = 0
    img_size = 256

In [12]:
cfg = config

In [13]:
device = "cuda"

**player-player models, single model train 4 seeds with all data**

In [14]:
models_pair = []
for fold in [0,2,3,4]:#[5,6,7,8]
    ckpt_path = f'/kaggle/input/nfl-data/kaggle/kaggle/r50ir_csn_c15_m1_d2_all_last_f{fold}.pth'
    model = NModel(cfg)
    checkpoint = torch.load(ckpt_path, map_location="cpu")
    model.load_state_dict(checkpoint)
    model.to(device)
    model.eval()
    models_pair.append(model)

**player-ground, single model train 3 seeds with all data (config 11)**

In [15]:
models_g_base = []
for fold in [0,1,2]:#[6,7,8]
    ckpt_path = f'/kaggle/input/nfl-data/kaggle/kaggle/r50ir_csn_c11_m1_d2_G_all_last_f{fold}.pth'
    model = NModel(cfg)
    checkpoint = torch.load(ckpt_path, map_location="cpu")
    model.load_state_dict(checkpoint)
    model.to(device)
    model.eval()
    models_g_base.append(model)

**player-ground, single model train 3 seeds with all data (config 15)**

In [16]:
models_g = []
for fold in [0,1,2]:
    ckpt_path = f'/kaggle/input/nfl-data/kaggle/kaggle/r50ir_csn_c15_m1_d2_G_all_last_f{fold}.pth'
    model_g = NModel(cfg)
    checkpoint = torch.load(ckpt_path, map_location="cpu")
    model_g.load_state_dict(checkpoint)
    model_g.to(device)
    model_g.eval()
    models_g.append(model_g)

**Load results of xgb preprocessing step and filter easy negative samples**

In [17]:
df = pd.read_csv('test_pair_xgb_v1.csv')
#filter easy negative player-player so that notebook would take less than 9 hours to run.
p_df = df[df.pred>0.002][['contact_id', 'frame']]#0.002

In [18]:
g_df = pd.read_csv('test_G.csv')
#filter easy negative player-ground 
g_df = g_df[g_df.pred>0.003][['contact_id', 'frame']]#0.0005 #0.002

In [19]:
df = pd.concat([p_df, g_df])

In [20]:
df['vid'] = df['contact_id'].apply(lambda x: '_'.join(x.split('_')[:2]))
df['step'] = df['contact_id'].apply(lambda x: int(x.split('_')[2]))
df['nfl_player_id_1'] = df['contact_id'].apply(lambda x: int(x.split('_')[3]))
df['nfl_player_id_2'] = df['contact_id'].apply(lambda x: x.split('_')[4])
vids = df.vid.unique()

In [21]:
df.tail()

,contact_id,frame,vid,step,nfl_player_id_1,nfl_player_id_2
4307,58172_003247_125_40656_G,1045.114,58172_003247,125,40656,G
4308,58172_003247_125_52521_G,1045.114,58172_003247,125,52521,G
4309,58172_003247_125_52939_G,1045.114,58172_003247,125,52939,G
4310,58172_003247_125_39008_G,1045.114,58172_003247,125,39008,G
4311,58172_003247_125_47912_G,1045.114,58172_003247,125,47912,G


In [22]:
#baseline helmet bboxes and tracking position is stored in dictionary for converience.
det_dict = np.load('det_dict.npy', allow_pickle=True).item()
trk_dict = np.load('trk_pos.npy', allow_pickle=True).item()

In [23]:
#define list of frame in each config
window_size_g = [-54, -48, -42, -36, -30, -24, -18, -13, -8, -4, -2, 0, 2, 4, 8, 13, 18, 24, 30, 36, 42, 48, 54]

# window_size_g_c15 = [-72, -66, -60, -54, -48, -42, -36, -30, -24, -18, -13, -8, -4, -2, 0, 2, 4, 8, 13, 18, 24, 30, 36]
# window_size_g_c15 = [x+6 for x in window_size_g_c15]
window_size_g_c15 = [-54, -48, -42, -36, -30, -24, -18, -13, -8, -4, -2, 0, 2, 4, 8, 13, 18, 24, 30, 36, 42, 48, 54]

frame_shift = 6
window_size_p = [-44, -37, -30, -24, -18, -13, -8, -4, -2, 0, 2, 4, 8, 13, 18, 24, 30, 37]
window_size_p = [x+frame_shift for x in window_size_p]

**Main inference code**

In [24]:
vid_dir = '/kaggle/input/nfl-player-contact-detection/test/'
results_g_base = []
results_g_pred = []
results_p_base = []
results_p_pred = []
for vid in tqdm(vids):
    print(f'process vid {vid}')
    #vid = vids[0]
    e_vid = vid + f'_Endzone'
    s_vid = vid + f'_Sideline'
    e_vid_path = f'{vid_dir}/{e_vid}.mp4'
    s_vid_path = f'{vid_dir}/{s_vid}.mp4'
    
    image_dict = {}
    image_dict1 = {}
    
    cap = cv2.VideoCapture(e_vid_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES,230)
    frame_count = 230
    while 1:
        ret, frame = cap.read()
        if not ret:
            break
        kk = f'{e_vid}_{frame_count}'
        image_dict[kk] = frame
        image_dict1[kk] = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_count +=1

    cap = cv2.VideoCapture(s_vid_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, 230)
    frame_count = 230
    while 1:
        ret, frame = cap.read()
        if not ret:
            break
        kk = f'{s_vid}_{frame_count}'
        image_dict[kk] = frame
        image_dict1[kk] = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_count +=1
        
    df_g = df[(df.vid==vid)&(df.nfl_player_id_2=='G')]
    df_p = df[(df.vid==vid)&(df.nfl_player_id_2!='G')]
    
    df_g_base, nona_g_base = predict_g(window_size_g, df_g, models_g_base, scale=4)
    if nona_g_base:
        #filter easy negative player-player so that notebook would take less than 9 hours to run.        
        df_g = df_g_base[df_g_base.pred>0.25]#0.01
    #df_g_base, nona_g_base = predict_g(window_size_g, df_g, models_g_base, scale=4)
        
    df_g_pred, nona_g_pred = predict_g(window_size_g_c15, df_g, models_g, scale=4.1)# 4.5    
    
    #df_p_base, nona_p_base = predict_pair(window_size_p, df_p, models_pair_base)
    #if nona_p_base:
    #    df_p = df_p_base[df_p_base.pred>0.01]
    df_p_pred, nona_p_pred = predict_pair(window_size_p, df_p, models_pair)
    
    if nona_p_pred:
        results_p_pred.append(df_p_pred)
        
    if nona_g_base:
        results_g_base.append(df_g_base)
    if nona_g_pred:
        results_g_pred.append(df_g_pred)
        
    gc.collect()

  0%|          | 0/2 [00:00<?, ?it/s]

process vid 58168_003392


/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/105 [00:00<?, ?it/s]

(834, 1) (834, 6)


...

(884, 1) (884, 6)


  0%|          | 0/80 [00:00<?, ?it/s]

(639, 1) (639, 7)


  0%|          | 0/148 [00:00<?, ?it/s]

(1178, 1) (1178, 6)


In [25]:
df_g_base = pd.concat(results_g_base)
df_g_pred = pd.concat(results_g_pred)

#df_p_base = pd.concat(results_p_base)
df_p_pred = pd.concat(results_p_pred)

**XGB post processing player-ground**

In [26]:
import xgboost as xgb

In [27]:
def get_oof(df_pred):
    pred_step_dict = {}
    for i, row in df_pred.iterrows():
        c_id = row['contact_id']
        pred = row['pred']
        gk, gp, st, idx1, idx2 = c_id.split('_')
        idx = f'{gk}_{gp}_{idx1}_{idx2}_{st}'
        pred_step_dict[idx] = row['pred']

    return pred_step_dict

def feature_engineering_G():
    train_df = pd.read_csv('test_G.csv')
    train_df['step'] = train_df['contact_id'].apply(lambda x: int(x.split('_')[2]))
    train_df['vid'] = train_df['contact_id'].apply(lambda x: '_'.join(x.split('_')[:2]))
    train_df['nfl_player_id_1'] = train_df['contact_id'].apply(lambda x: int(x.split('_')[3]))
    train_df['nfl_player_id_2'] = 'G'

    pred_step_dict1 = get_oof(train_df)

    pred_step_dict = get_oof(df_g_base)
    pred_step_dict2 = get_oof(df_g_pred)
    
    results = []
    for _, row in tqdm(train_df.iterrows()):
        idx1 = row['nfl_player_id_1']
        idx2 = row['nfl_player_id_2']
        vid = row['vid']
        step = row['step']

        item = {'contact_id':row['contact_id'], 'step':row['step'], 'frame': row['frame']}
        item['nfl_player_id_1'] = row['nfl_player_id_1']
        item['nfl_player_id_2'] = row['nfl_player_id_2']
        item['vid'] = row['vid']

        for i in range(-15,15):
            this_idx = f'{vid}_{idx1}_{idx2}_{step+i}'
            prob = 0
            weight = 0
            if this_idx in pred_step_dict:
                prob += pred_step_dict[this_idx]
                weight += 1

            if this_idx in pred_step_dict2:
                prob += pred_step_dict2[this_idx]
                weight += 1

            if weight > 0:
                item[f'prob_{i}'] = prob/weight
            else:
                item[f'prob_{i}'] = np.nan

            if this_idx in pred_step_dict1:
                item[f'prob1_{i}'] = pred_step_dict1[this_idx]
            else:
                item[f'prob1_{i}'] = np.nan

        results.append(item) 

    train_df = pd.DataFrame(results)

    results = []
    k=0
    for _, row in tqdm(train_df.iterrows()):
        item = {}
        for i in range(-15,15):
            if i>-10 and i < 10:
                item[f'prob_{i}'] = row[f'prob_{i}']
                item[f'prob1_{i}'] = row[f'prob1_{i}']
            item[f'prob3_{i}'] = 0.85*row[f'prob_{i}']  + 0.15*row[f'prob1_{i}']

        if k==0: feature_cols = list(item.keys())
        k+=1

        item['contact_id'] = row['contact_id']    

        results.append(item)

    train_df = pd.DataFrame(results)
    return train_df, feature_cols

train_df_g, feature_cols = feature_engineering_G()

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [28]:
x_valid = train_df_g[feature_cols]
dvalid = xgb.DMatrix(x_valid)
for fold in [0,1,2]:# [0,1,2,3,4]
    model_path = f'/kaggle/input/nfl-data/xgb_models/xgb_models/xgb_fold{fold}_xgb_2nd.model'
    model = xgb.Booster()
    model.load_model(model_path)
    
    if fold==0:
        pred_i = model.predict(dvalid) 
    else:
        pred_i += model.predict(dvalid)
    print(pred_i.shape)
    
pred_i = pred_i/3#5

(4312,)
(4312,)
(4312,)


In [29]:
train_df_g['pred'] = pred_i
train_df_g = train_df_g[['contact_id', 'pred']]

In [30]:
train_df_g[train_df_g.pred>0.5].shape

(281, 2)

**XGB post processing player-player**

In [31]:
def feature_engineering_p():
    train_df = pd.read_csv('test_pair_xgb_v1.csv')
    train_df['step'] = train_df['contact_id'].apply(lambda x: int(x.split('_')[2]))
    train_df['vid'] = train_df['contact_id'].apply(lambda x: '_'.join(x.split('_')[:2]))
    train_df['nfl_player_id_1'] = train_df['contact_id'].apply(lambda x: int(x.split('_')[3]))
    train_df['nfl_player_id_2'] = train_df['contact_id'].apply(lambda x: int(x.split('_')[4]))
    
    print(train_df.shape)

    #pred_step_dict = get_oof(df_p_base)
    pred_step_dict2 = get_oof(df_p_pred)
    
    xgb_df = pd.read_csv('test_pair_xgb_v1.csv')
    xgb_dict = get_oof(xgb_df)

    results = []
    for _, row in tqdm(train_df.iterrows()):
        idx1 = row['nfl_player_id_1']
        idx2 = row['nfl_player_id_2']
        step = row['step']
        vid = row['vid']

        item = {'contact_id':row['contact_id']}
        item['nfl_player_id_1'] = row['nfl_player_id_1']
        item['nfl_player_id_2'] = row['nfl_player_id_2']

        for i in range(-10,10):
            this_idx = f'{vid}_{idx1}_{idx2}_{step+i}'
            prob = 0
            weight = 0
            if this_idx in pred_step_dict2:
                prob += pred_step_dict2[this_idx]
                weight += 1

            if weight > 0:
                item[f'prob_{i}'] = prob/weight
            else:
                item[f'prob_{i}'] = np.nan

            if this_idx in xgb_dict:
                item[f'prob1_{i}'] = xgb_dict[this_idx]
            else:
                item[f'prob1_{i}'] = np.nan

        results.append(item) 

    train_df = pd.DataFrame(results)

    results = []
    k=0
    for _, row in tqdm(train_df.iterrows()):
        item = {}
        for i in range(-10,10):
            item[f'prob4_{i}'] = 0.8*row[f'prob_{i}'] + 0.2*row[f'prob1_{i}']

        if k==0: feature_cols = list(item.keys())
        k+=1

        item['contact_id'] = row['contact_id']

        results.append(item)

    train_df = pd.DataFrame(results)
    return train_df, feature_cols

train_df_p, feature_cols = feature_engineering_p()

(7163, 8)


0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [32]:
x_valid = train_df_p[feature_cols]
dvalid = xgb.DMatrix(x_valid)
for fold in [0,2,3,4]:#[0,1,2,3,4]
    model_path = f'/kaggle/input/nfl-data/xgb_models/xgb_models/xgb_not_fold{fold}_xgb_2nd.model'
    model = xgb.Booster()
    model.load_model(model_path)
    
    if fold==0:
        pred_i = model.predict(dvalid) 
    else:
        pred_i += model.predict(dvalid)
    print(pred_i.shape)
pred_i = pred_i/4#5


(7163,)
(7163,)
(7163,)
(7163,)


In [33]:
train_df_p['pred'] = pred_i*0.95# keep 95% of predictions
train_df_p = train_df_p[['contact_id', 'pred']]

In [34]:
print(train_df_p.shape)
print(train_df_p[train_df_p.pred>0.38].shape)

(7163, 2)
(595, 2)


**Merge results and save to submission.csv**

In [35]:
df = pd.concat([train_df_p, train_df_g])

In [36]:
# at threshold 0.38 xgb models shown best score.
df['contact'] = 1*(df['pred']>0.38)
df = df[['contact_id', 'contact']].reset_index(drop=True)

In [37]:
sub = pd.read_csv('/kaggle/input/nfl-player-contact-detection/sample_submission.csv')

In [38]:
df1 = pd.concat([df, sub])

In [39]:
df1.head(3)

,contact_id,contact
0,58168_003392_0_38590_41944,0
1,58168_003392_0_38590_47944,0
2,58168_003392_0_38590_44822,0


In [40]:
# column ['contact_id'] for identifying duplicates. keep: {‘first’, ‘last’, False}, default ‘first’
df1 = df1.drop_duplicates(subset=['contact_id'])

In [41]:
df1.to_csv('submission.csv', index=False)

In [42]:
#EOF